## The Implementation

I put the [theano](http://deeplearning.net/software/theano/) RNN model and training in two different classes: ```embedding_model``` and ```word_embedder```. An ```embedding_model``` builds and compiles theano functions for the RNN network, the loss, and the stochastic gradient descent ([SGD](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)) update. A ```word_embedder``` is responsible for running training minibatches, initialized from data or pickle files.

### The basics

Let's get started with imports and globals:

In [1]:
import numpy as np

import theano
from theano import tensor as T


floatX = theano.config.floatX


# the activation function and softmax
__f = T.nnet.sigmoid
__y = T.nnet.softmax

I put the basic units of the [model](the_model.ipynb) in three functions: ```rnn```, ```compute_network_outputs```, and ```compute_mean_log_lklyhd_outputs```.  A single neuron of the model is quite simple and can be taken care of in one line of code (ignoring the comments and function definition):

In [2]:
def rnn(i_t,s_tm1,U,W,b) : 
    """ A single neuron in the recursive network
    
        Input
        
        i_t   - one-of-n index encoding
        s_tm1 - output from the previous stage of the rnn
        U     - weight matrix for x_t
        W     - weight matrix for s_tm1
        b     - bias vector
        
        Output
        
        s_t   - output of the t-th stage of the rnn
    """
    
    return __f(U[:,i_t] + W.dot(s_tm1) + b)

Originally I had this function take in a vector ```x``` in place of the index ```i_t```. With the one-of-n encoding, ```x``` will always be a standard Euclidean basis vector consisting of a one at some index (```i_t```) and zeros everywhere else. So ```x``` amounts to a wildly inefficient encodding of the index ```i_t```. In fact this oversight made training cripplingly slow.

The full network is produced by looping over a vector of one-of-n indices (representing a sentence) in the forward and backward directions summing the results, multipling by another weight matrix and passing that to the soft max function:

In [3]:
def compute_network_outputs(i,s0,V,U,W,b) : 
    """ Builds the recursive network using theano's scan iterator. This 
        is a bidirectional rnn where we simply add the outputs of the 
        rnns in both directions. 
        
        Input
        
        i    - array of one-of-n indices representing a sentence
        s0   - initial output of the rnn
        V    - weight matrix for softmax step
        U    - weight matrix for the input to the recursive neuron
        W    - weight matrix for the previous output of the recursive neuron
        b    - bias vector
        
        Output
        
        y    - uncompiled symbolic theano representation of the rnn
    """
    
    fwrd_rslt,_ = theano.scan(rnn,
                              sequences = [i],
                              outputs_info = [s0],
                              non_sequences = [U,W,b])
    bwrd_rslt,_ = theano.scan(rnn,
                              sequences = [i[::-1]],
                              outputs_info = [s0],
                              non_sequences = [U,W,b])
    
    return __y(T.dot(fwrd_rslt + bwrd_rslt, V.T))

Looping in theano is handled by the [```scan```](http://deeplearning.net/software/theano/library/scan.html) function. I'd like to pause here for a moment and reflect on how cool theano is. In all honesty when I started playing around with theano it was somewhat revelatory. I am a technical person and I use Python to solve technical problems from all over the mathematical map every day at work. Many are optimization problems. And I have taken for granted the notion that the real meat of an optimization problem is programming the objective, and it's gradient and Hessian. For neural networks this translates to programming backpropagation. As far as theano is concerned however that way of thinking is incredibly naive (and I evidenty was). In theano the work, such as it is, is in setting up the function model graph, and whatever update you may need for your favorite flavor of SGD. Then theano takes care of the rest with exact (symbolic) differentiation. I'm getting a bit over-worked here but suffices to say that I think theano is cool.

Returning to the code at hand. It's probably easiest to communicate what's going on in the calls to the ```scan``` function by translating one of them into a regular old Python loop. First the call to scan:

```python
fwrd_rslt,_ = theano.scan(rnn,
                          sequences = [i],
                          outputs_info = [s0],
                          non_sequences = [U,W,b])
```

As a Python loop this would read

```python
data = np.zeros((len(i)+1,len(s0)))
data[0] = s0
for j,i_t in enumerate(i) : 
    data[j+1] = rnn(i_t,data[j],U,W,b)
fwrd_rslt = data[1:]
```

So in this case the ```scan``` function is mapping the function ```rnn``` across the vector ```i``` with normal arguments ```U```, ```W```, and ```b``` and a special argument ```s_tm1``` whose type is the same as ```s0``` and whose value is the output of the call to ```rnn``` from the previous iteration, initialized with ```s0```.

What about the second output of ```scan``` that in this case we are saving in the the throw-away variable '```_```'? As I understand it, that output is a dictionary of 'updates' for theano shared variables. Theano shared variables allow theano functions to have internal states, like counting variables or, as will be the case for us, weight matrices. A given function may update the value of a shared variable and theano needs to keep track of that in a special way. In the code above, there will be shared variables (```V```, ```U```, ```W```, and ```b```) but network evaluation does not update their values so the updates dictionary returned by ```scan``` is empty.

For the loss we need to map ```compute_network_outputs``` across the test sentences (matrix of one-of-n indices) to produce the probabilities of the vocabulary words occuring at the various locations in sentences, and take the mean of the log of a requested collection of these probabilities. The code is likely a bit more clear:

In [4]:
def compute_mean_log_lklyhd_outputs(I,J,s0,V,U,W,b) : 
    """ Builds theano symbolic representation of the mean log likelyhood
        loss function for the embedding model. 
        
        Input
    
        I    - matrix of one-of-n indices representing input sentences
        J    - matrix of one-of-n indices representing target sentences
        s0   - initial output of the rnn
        V    - weight matrix for softmax step
        U    - weight matrix for the input to the recursive neuron
        W    - weight matrix for the previous output of the recursive neuron
        b    - bias vector
        
        Output
        
        uncompiled symbolic theano representation of mean log likelyhood loss
    """
        
    # the function to scan over - it just collects the log likelyhood of 
    # the positions indicated by j given i and the weight matrices
    def scan_f(i,j,s0,V,U,W,b) : 
        
        outs = compute_network_outputs(i,s0,V,U,W,b)
        
        return T.log(theano.scan(lambda j_t,o : o[j_t],
                                 sequences = [j,outs])[0]) 
    
    batch_outputs,_ = theano.scan(scan_f,
                                  sequences = [I,J],
                                  non_sequences = [s0,V,U,W,b])
    
    return T.mean(batch_outputs)

### ```embedding_model```

The ```embedding_model``` object collects the theano symbolic representations for the network, the loss, the gradient and the SGD update, and lets you compile (call theano's ```function``` method) each as needed. Each embedding model instance will depend on the size of the problem vocabulary, ```n```, and the dimension of the embedding space, ```k```. The version of this class discussed here is a slighly pared down version of the one in the repository. Here is the ```___init___``` method:

In [5]:
class embedding_model : 
    """ Bidirectional RNN for experimenting with word embeddings. The model is
        the following
           
        s_+(t) = f(Uw(t) + Ws_+(t-1) + b)
        s_-(t) = f(Uw(m-t) + Ws_-(t-1) + b)
        y(t) = g(Vs_+(t) + Vs_-(t))
        
        and
        
        f(z) = 1 / (1 + e^{-z})
        g(z_m) = e^{z_m} / sum e^{z_k}
        
        This is just a bidirectional version of the model that appears in 
        https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf
    """
    
    def __init__(self,
                 n,   # dimension of inputs
                 k,   # dimension of embedding space
                 seed = None,
                 s0 = None,
                 int_dtype = 'int64',
                 float_dtype = floatX
                 ) : 
        """ Model initializer
        
            Inputs
            
            n     - dimension of the inputs at each recursive layer
            k     - dimension of the embedding space
            seed  - optional seed for random initialization of weight matrices
            s0    - optional initial output to start netword recursion
            dtype - float type for computations
        """
        
        self.n = n
        self.k = k
        self.seed = seed
        self.int_dtype = int_dtype
        self.float_dtype = float_dtype
        
        # seed the random generator if requested
        if seed is not None : 
            np.random.seed(seed)
            
        # collect s0
        if s0 is None : 
            self.s0 = theano.shared(np.zeros(k,dtype = self.float_dtype),name='s0')
        else : 
            self.s0 = theano.shared(np.array(s0,dtype = self.float_dtype),name='s0')
        
        # initialize the weights
        self.V = theano.shared(np.array(np.random.rand(n,k),
                                        dtype = self.float_dtype),name='V')
        self.U = theano.shared(np.array(np.random.rand(k,n),
                                        dtype = self.float_dtype),name='U')
        # initializing with full matrix can lead to vanishing gradients
        self.W = theano.shared(np.array(np.diag(np.random.rand(k)),
                                        dtype=self.float_dtype),name='W')
        self.b = theano.shared(np.array(np.random.rand(k),
                                        dtype = self.float_dtype),name='b')  
        
        # shared variables for mean gradient magnitudes
        self.m_dV_mag = theano.shared(np.array(0.,dtype = self.float_dtype),
                                      name='m_dV_mag')
        self.m_dU_mag = theano.shared(np.array(0.,dtype = self.float_dtype),
                                      name='m_dU_mag')
        self.m_dW_mag = theano.shared(np.array(0.,dtype = self.float_dtype),
                                      name='m_dW_mag')
        self.m_db_mag = theano.shared(np.array(0.,dtype = self.float_dtype),
                                      name='m_db_mag')
        
        # shared variables for computing the loss
        self.loss_accum = theano.shared(np.array(0.,dtype = self.float_dtype),
                                        name='loss_accum')
        self.loss_accum_i = theano.shared(np.array(0.,dtype = self.float_dtype),
                                          name='loss_accum_i')
        
        # compute the network
        self._compute_network_model()

In the ```__init__``` method, all the theano shared variables are initialized and then the ```_compute_network_model``` method is called. In addition to shared variables for the weights which will be updated in each SGD iteration during training, there are also variables for the mean, across SGD updates, of the gradient magnitudes of the weights (```m_d```*```_mag```), and variables for computing the loss. I've set up the loss in a way that allows it to be computed in a series of batches. This arose out of the silly way I originally implemented ```rnn``` (as mentioned above, with full one-of-n vector inputs) which led to Python kernel death if I passed all the test data to the loss function. With the implementation of ```rnn``` given above there is no need for loss evaluation across batches. 

Notice that I am initializing ```W```, the weights on the output from the previous recursive layer, with a random diagonal matrix. This is in response to the observation that ```W``` gradients vanish if initialization is done naively with a dense random matrix.

The ```_compute_network_model``` method builds all the required theano function graphs. Note that in order to split the definition of the class up across several Jupyter notebook cells I am defining the following methods as Python functions and then binding them to the ```embedding_model``` class (which is the content of the last line of the following cell).

In [6]:
def _compute_network_model(self) : 
    """ Build the network, loss, grad_loss and sgd_update theano functions.
        More work than is strictly nessecary is done here as the only thing
        that is really needed in order to run sgd (stochastic gradient 
        descent) is the sgd_update function. The network, loss and grad_loss
        functions are compiled since this is experimental code.
    """

    # build the network
    self.i = T.vector('i',dtype = self.int_dtype)

    self.network_outputs = compute_network_outputs(self.i,self.s0,self.V,
                                                   self.U,self.W,self.b)


    # build mean log likelyhood loss

    # variables for a batch of sentences
    self.I = T.matrix('I',dtype = self.int_dtype)
    self.J = T.matrix('J',dtype = self.int_dtype) # for embedding I = J

    self.loss_outputs = compute_mean_log_lklyhd_outputs(self.I,self.J,
                                                        self.s0,self.V,
                                                        self.U,self.W,
                                                        self.b)

    # set up the accumulator for computing the loss in batches

    n_minibatch = T.cast(self.I.shape[0],self.float_dtype)
    loss_accum_ipnm = self.loss_accum_i + n_minibatch

    self.loss_updates = ((self.loss_accum,
                          (self.loss_outputs*n_minibatch/loss_accum_ipnm
                           + (self.loss_accum 
                             * self.loss_accum_i/loss_accum_ipnm))),
                         (self.loss_accum_i,loss_accum_ipnm))

    # get the gradient of the loss

    (self.dV,
     self.dU,
     self.dW,
     self.db) = theano.grad(self.loss_outputs,
                            [self.V,self.U,self.W,self.b])

    # get the gradient magnitudes

    self.dV_mag = T.sqrt(T.sum(self.dV*self.dV))
    self.dU_mag = T.sqrt(T.sum(self.dU*self.dU))
    self.dW_mag = T.sqrt(T.sum(self.dW*self.dW))
    self.db_mag = T.sqrt(T.sum(self.db*self.db))

    # get the sgd update function

    # this is the learning parameter
    self.eta = T.scalar('eta',dtype = self.float_dtype)

    # also including a running average of the gradient magnitudes

    self.sgd_i = T.scalar('sgd_i',dtype = self.float_dtype)

    dV_mag_accum = (self.dV_mag/(self.sgd_i+1.)
                        + self.m_dV_mag*(self.sgd_i/(self.sgd_i+1.)))
    dU_mag_accum = (self.dU_mag/(self.sgd_i+1.) 
                        + self.m_dU_mag*(self.sgd_i/(self.sgd_i+1.)))
    dW_mag_accum = (self.dW_mag/(self.sgd_i+1.) 
                        + self.m_dW_mag*(self.sgd_i/(self.sgd_i+1.)))
    db_mag_accum = (self.db_mag/(self.sgd_i+1.) 
                        + self.m_db_mag*(self.sgd_i/(self.sgd_i+1.)))

    # adding here since we are taking a max of the loss - accumulators
    # do not include the latest values
    self.sgd_updates = ((self.V,self.V + self.eta*self.dV),
                        (self.U,self.U + self.eta*self.dU),
                        (self.W,self.W + self.eta*self.dW),
                        (self.b,self.b + self.eta*self.db),
                        (self.m_dV_mag,dV_mag_accum),
                        (self.m_dU_mag,dU_mag_accum),
                        (self.m_dW_mag,dW_mag_accum),
                        (self.m_db_mag,db_mag_accum))

    # pointers for the compiled functions
    self.network = None
    self.loss = None
    self.grad_loss = None
    self.sgd_update = None
    self.sgd_update_w_loss = None

# bind the method to the class - this is just so the class definition
# can be broken up across several Jupyter notebook cells
embedding_model._compute_network_model = _compute_network_model

As expected, the network and loss graphs are built in ```compute_network_outputs``` and ```compute_mean_log_lklyhd_outputs```. Theano's ```grad``` function makes computing derivatives trivial. The efforts I undertake with the  *```_accum``` variables might look abit cumbersome, but all I am doing is multiplying by the number of summands in the previous accumulated mean, adding the new summands and dividing everything by the new total. For the loss accumulation I am keeping the number of summands in the last loss evaluation in the variable ```self.loss_accum_i```. For the mean gradient magnitude accumulation the corresponding quantity is the number of minibatches so far, I am keeping that number in ```self.sgd_i```. 

One other thing deserves note: since we are doing maximum likelyhood, the SGD update *adds* the scaled gradient to the weight matrices. Typically SGD is stated for a minimization problem, in which case the right thing to do is subtract a scaled version of the gradient. So in our case we are really doing stochastic gradient ascent. 

Next we have the methods that call theano's ```function``` method to actually compile the graphs so data can be passed through them. Below I've only included compile methods for the network, the loss, and the SGD update. The code in the repository also has methods for compiling the gradient of the loss and a version of the SGD update that in addition to running the update also returns the loss. As before, in order to split the class definition across multiple Jupyter cells the following methods are defined as Python functions and then bound to the ```embedding_model``` class.

In [7]:
def compile_network(self) : 
    """ Compile the network  
    """
    if self.network is not None : 
        return

    self.network = theano.function(inputs = [self.i],
                                   outputs = self.network_outputs)

# bind the method to the class
embedding_model.compile_network = compile_network

def compile_loss(self) : 
    """ Compile the loss
    """

    if self.loss is not None : 
        return

    self.loss = theano.function(inputs = [self.I,self.J],
                                outputs = self.loss_outputs,
                                updates = self.loss_updates)

# bind the method to the class
embedding_model.compile_loss = compile_loss
        
def compile_sgd_update(self) : 
    """ Compile SGD update function
    """

    if self.sgd_update is not None : 
        return

    self.sgd_update = theano.function(inputs = [self.I,self.J,
                                                self.eta,self.sgd_i],
                                      outputs = [],
                                      updates = self.sgd_updates)

# bind the method to the class
embedding_model.compile_sgd_update = compile_sgd_update

Theano's ```function``` method takes in the inputs and outputs of the desired function, together with any desired updates on shared varialbes. Evaluating the network does not involve shared variable updates. In the SGD update however the whole point is to update the weights of the model, so in ```compile_sgd_update``` there are no outputs in the call to ```function```, only inputs and updates.

Now we have enough in place to initialize an example model and play around with some fake data:

In [8]:
# initialize a model
n_vocab = 5
dim_embedding = 2
seed = 12345
model = embedding_model(n_vocab,dim_embedding,seed)

# compile the network
model.compile_network()

# make some dummy data
n_words_in_sent = 10
sentence = np.array([np.random.randint(n_vocab) for i in range(n_words_in_sent)])

# evaluate the network on the dummy data
model.network(sentence)

array([[ 0.18455518,  0.05965285,  0.17996989,  0.3272838 ,  0.24853828],
       [ 0.18423066,  0.05006433,  0.1724503 ,  0.34385194,  0.24940277],
       [ 0.18613941,  0.04683204,  0.16852475,  0.35007937,  0.24842442],
       [ 0.18665375,  0.04682157,  0.16826569,  0.35010207,  0.24815692],
       [ 0.18246037,  0.04999981,  0.17327421,  0.34394022,  0.25032539],
       [ 0.18199794,  0.05041981,  0.17388096,  0.34314667,  0.25055461],
       [ 0.18616735,  0.04726935,  0.16892835,  0.34921873,  0.24841623],
       [ 0.1863764 ,  0.04651053,  0.16810058,  0.35071763,  0.24829486],
       [ 0.18460649,  0.04865383,  0.17098595,  0.34652341,  0.24923032],
       [ 0.18221862,  0.04946176,  0.17290928,  0.34494665,  0.2504637 ]])

Let's make some fake data and evaluate the loss:

In [9]:
# compile the loss
model.compile_loss()

# make some fake training data
n_sentences = 100
in_sentences = np.array([[np.random.randint(n_vocab) 
                              for i in range(n_words_in_sent)]
                                                for j in range(n_sentences)])
out_sentences = np.array([[np.random.randint(n_vocab) 
                              for i in range(n_words_in_sent)]
                                                for j in range(n_sentences)])

# evaluate the loss on the fake data
model.loss(in_sentences,out_sentences)

array(-1.798485079447346)

And let's run a couple SGD updates:

In [10]:
# compile the SGD update
model.compile_sgd_update()

# do SGD on the fake data 

minibatch_size = 50
eta = 0.001 # learning rate

i_n = int(np.ceil(n_sentences/minibatch_size))
for i in range(i_n) :

    # get the batch data
    in_batch = in_sentences[i*minibatch_size:(i+1)*minibatch_size]
    out_batch = out_sentences[i*minibatch_size:(i+1)*minibatch_size]
    
    # do the update and get the loss - 
    model.sgd_update(in_batch,out_batch,eta,i)
    
    print("minibatch", i+1)
    print("mean dV mag =",model.m_dV_mag.get_value())
    print("mean dU mag =",model.m_dU_mag.get_value())
    print("mean dW mag =",model.m_dW_mag.get_value())
    print("mean db mag =",model.m_db_mag.get_value())
    if i != i_n-1 : print("\n")

minibatch 1
mean dV mag = 0.5222211706893597
mean dU mag = 0.022116767224227115
mean dW mag = 0.04942633568319734
mean db mag = 0.04899231231942744


minibatch 2
mean dV mag = 0.5136943284427516
mean dU mag = 0.023094645546331213
mean dW mag = 0.049932791733365636
mean db mag = 0.0504770132400983


So that's the implementation of the RNN model. There is at least one thing that I think could be improved: I would like to factor out the SGD update. Philosophically, the gradient descent update does not belong in the model since it is not part of the RNN. Practically, if the SGD update is factored out of the model then supporting other flavors of SGD (AdaGrad, Adam, etc.) would be more natural.

### ```word_embedder```

The ```word_embedder``` object deals with reading sentence data, reading and writing training states, and with training. Sentence data is read from a numpy npz file, from which unique words are extracted and sorted by frequency. The top ```n_vocabulary-2``` most frequent words are taken as the vocabulary, together with two special tokens, one indicating the end of a sentence and another indicating an unknown word. The sentences are then translated into one-of-n indices with respect to the vocabulary and separated into training and testing sets. 

As with the ```embedding_model``` class given above, the ```word_embedder``` class discussed below is a stripped-down version of the one in the repository. Namely the one given here does not include the methods for reading/writing a training state from/to a pickle file. 

I put sentence reading and one-of-n encoding into the following two functions:

In [11]:
from collections import Counter

def read_sentences(path,n_vocabulary) : 
    """ Read sentences from the disk and collect the words, word counts
        and vocabulary. 
        
        Sentence data is assumed to start with a '<s>' token and end with 
        a list of '<e>' tokens. The top n_vocabulary most common words are 
        extracted from the sentence data to form the vocabulary. Of these, the
        least common word is replaced with a '<u>' (unknown) token and if
        '<e>' is not present in the vocabulary the second least common word 
        is replaced with '<e>'.
        
        Inputs : 
            
        path         - path to the sentence .npz file
        n_vocabulary - top n_vocabulary most common words will form the vocab
        
        Outputs : 
            
        sentences    - array of sentences read from the disk
        words        - all the words present in the sentences
        counts       - the number of occurances of each word in the data
        vocabulary   - the vocabulary
    """
    
    if n_vocabulary < 2 : 
        raise ValueError('Need more than two words in vocabulary')
    
    # load the sentences - not including start symbol
    sentences = np.load(path)['arr_0'][:,1:]

    # get the words and word counts
    words,counts = np.unique(sentences,return_counts = True)
    
    # get the sort map
    sort_map = np.argsort(counts)
    
    # collect the vocabulary
    
    vocabulary = np.zeros(n_vocabulary,dtype = words.dtype)
    vocabulary = words[sort_map][-n_vocabulary:][::-1]

    if '<e>' not in vocabulary : 
        vocabulary[-2] = '<e>'
        vocabulary[-1] = '<u>'
    else : 
        vocabulary[-1] = '<u>'
        
    return (sentences,words,counts,vocabulary)
        

def get_one_of_n_indices(vocabulary,sentences,
                         dtype = np.int16) : 
    """ Translate the sentence data into index data - each word is replaced
        by its index in the vocabulary array
        
        Inputs : 
            
        vocabulary   - array of vocabulary words
        sentences    - array of sentences
        dtype        - signed integer type for the output array
        
        Outputs : 
            
        index array  - array in which the words in each sentence in the
                       sentences array are replaced by their corresponding
                       index in the vocabulary array
    """
      
    if '<u>' != vocabulary[-1] : 
        raise ValueError("Last member of vocabulary should be '<u>' token")
    
    n_vocab = len(vocabulary)

    if np.array(n_vocab,dtype=dtype) != n_vocab : 
        raise ValueError("Provided dtype cannot represent size of vocabulary")
    if np.array(-1,dtype=dtype) != -1 : 
        raise ValueError("Provided dtype must be signed")
    
    # using counter since if a key is not in a counter
    # it returns zero, so unknown tokens will return zero
    vocab_map = Counter(dict((token,i+1) 
                                for i,token in enumerate(vocabulary[:-1])))

    return np.array([[vocab_map[token]-1 for token in sentence]
                                     for sentence in sentences],dtype = dtype)

A ```word_embedder``` object should be initialized from a sentence npz or from a pickled training state. The following ```__init__``` method is used inside those operations:

In [12]:
import logging
import time

# print all logging statements to stdout
logging.basicConfig(format='%(levelname)s:%(message)s',
                    level=logging.DEBUG)


class word_embedder : 
    """ Class for training a word embedding RNN
    """
    
    def __init__(self,
                 vocabulary,
                 training_data,
                 test_data,
                 model,
                 minibatch_size,
                 eval_test_loss_stride,
                 n_test_loss_batches,
                 seed = None,
                 logger = logging
                 ) :
        """ word_embedder initializer
        
            Inputs : 
                
            vocabulary            - words in the vocabulary, including '<e>'
                                    and '<u>' tags (end-of-sentence, and unknown)
            training_data         - training data as one-of-n indices to the
                                    vocabulary array
            test_data             - test data as one-of-n indices to the 
                                    vocabulary array
            model                 - embedding model
            minibatch_size        - number of sentences to use in a minibatch
            eval_test_loss_stride - number of minibatches between loss test
                                    loss evaluations
            n_test_loss_batches   - number of batches over which to accumulate 
                                    test loss
            seed                  - seed for random shuffle after each epoch,
            logger                - logging object
        """
        
        self.vocabulary = vocabulary
        self.n_vocabulary = len(vocabulary)
        self.training_data = training_data
        self.training_data_size = len(training_data)
        self.training_indices = np.arange(self.training_data_size)
        self.test_data = test_data
        self.test_data_size = len(self.test_data)
        self.model = model
        self.minibatch_size = minibatch_size
        self.eval_test_loss_stride = eval_test_loss_stride
        self.n_test_loss_batches = n_test_loss_batches
        
        self.test_loss_batch_size = self.test_data_size
        self.test_loss_batch_size /= self.n_test_loss_batches
        self.test_loss_batch_size = int(np.ceil(self.test_loss_batch_size))
 
        self.seed = 0
        if seed is not None : 
            self.seed = seed
        np.random.seed(self.seed)
        
        self.logger = logger
        
        # a list of pairs of the form (minibatch number, loss)
        self.test_loss = [(0,-np.inf)]
        
        self.best_test_loss = (0,-np.inf)
        
        # the total number of minibatches run so far
        self.minibatch_i = 0
        
        # list of indices giving minibatch counts at which training sessions
        # have been run
        self.training_minibatches = [0]
        
        # list of times for training sessions
        self.training_times = [0.]
        
        self.best_V = self.model.V.get_value()
        self.best_U = self.model.U.get_value()
        self.best_W = self.model.W.get_value()
        self.best_b = self.model.b.get_value()
        
        # mean gradient magnitudes - recorded every minibatch iteration
        self.m_dV_mag = []
        self.m_dU_mag = []
        self.m_dW_mag = []
        self.m_db_mag = []

Note that ```__init__``` does not build an RNN model, it expects one as input. RNN model building occurs in the following classmethod which implements initialization from a sentence npz file:

In [13]:
def init_from_sentences_on_disk(cls,
                                sentences_path,
                                n_vocabulary,
                                n_training_data,
                                embedding_space_dim,
                                minibatch_size,
                                eval_test_loss_stride,
                                n_test_loss_batches,
                                nof1_dtype = np.int16,
                                float_dtype = floatX,
                                shuffle_seed = None,
                                model_seed = None,
                                embedder_seed = None,
                                logger = logging
                                ) :
    """ word_embedder initialization from sentence data on disk

        Inputs : 

        sentences_path        - file path to sentences .npz: sentences
                                should start with '<s>' tag and end with 
                                one or more '<e>' tags
        n_vocabulary          - vocabulary will be top n_vocabulary most
                                common words in sentence data: will always
                                include '<e>' and '<u>' tokens
        n_training_data       - number of sentences in the training data
        embedding_space_dim   - number of rows in input weights of RNN model 
        minibatch_size        - number of sentences in a minibatch
        eval_test_loss_stride - number of minibatches between test loss
                                evaluations
        n_test_loss_batches   - number of batches over which to accumulate 
                                test loss 
        nof1_dtype            - integer type to use in storing one-of-n
                                index arrays
        float_dtype           - numeric type for use in rnn model weights
        shuffle_seed          - seed for random shuffle of sentence data
        model_seed            - seed for rnn model weight initialization
        embedder_seed         - seed for random shuffle after each epoch
        logger                - logging object

        Output :

        word_embedder instance
    """

    start = time.perf_counter()

    # read the sentences from the disk
    logger.info('reading sentences from %s'%sentences_path)
    (sentences,
     words,
     counts,
     vocabulary) = read_sentences(sentences_path,n_vocabulary)

    # get the one of n indices
    logger.info('computing one-of-n indices')
    data = get_one_of_n_indices(vocabulary,sentences,dtype = nof1_dtype)

    # shuffle the data

    if shuffle_seed is not None : 
        np.random.seed(shuffle_seed)

    np.random.shuffle(data)

    # partition the training and test data
    training_data = data[:n_training_data]
    test_data = data[n_training_data:]

    # initialize the rnn model
    logger.info('initializing embedding model')
    model = embedding_model(n_vocabulary,
                            embedding_space_dim,
                            model_seed,
                            int_dtype = nof1_dtype,
                            float_dtype = float_dtype)

    elapsed = time.perf_counter() - start
    logger.info('init from sentences elapsed time = %fs'%elapsed)

    return cls(vocabulary,
               training_data,
               test_data,
               model,
               minibatch_size,
               eval_test_loss_stride,
               n_test_loss_batches,
               embedder_seed,
               logger)

# add the function to the class as a classmethod - this is
# only needed because I am defining the class across
# several Jupyter notebook cells
word_embedder.init_from_sentences_on_disk = classmethod(init_from_sentences_on_disk)

Here, the sentences are read, the one-of-n encoding and vocabulary are computed, the data is partitioned into training and test sets, and the RNN model is initialized. For fun I also included a random shuffle on the sentence data. 

I've mentioned several times that my first implementation of ```rnn``` was inefficient and led me to build in support for loss evaluation over batches. In light of the fact that accumulation turned out not to be necessary you may be wondering why I didn't get rid of it altogether, and avoid all this handwringing. My reasoning is that something like it may in fact be necessary when I start stacking RNN's up to build deep networks since inner layers will have to deal with dense vector inputs. 

Well without further ado, here is the loss accumulation method:

In [14]:
def accumulate_loss(self) : 
    """ Accumulate the value of the loss function on the test data across
        self.n_test_loss_batches batches

        Output : 

        value of the loss function on the test data      
    """

    self.logger.info("accumulating loss")

    self.model.loss_accum.set_value(0.)
    self.model.loss_accum_i.set_value(0.)

    for j in range(self.n_test_loss_batches) : 

        self.logger.info("batch %d of %d"%(j+1,self.n_test_loss_batches))

        j0 = j*self.test_loss_batch_size
        j1 = (j+1)*self.test_loss_batch_size

        batch = self.test_data[j0:j1]

        self.model.loss(batch,batch)

    self.logger.info("loss accumulation complete")

    return self.model.loss_accum.get_value()

# bind the method to the class
word_embedder.accumulate_loss = accumulate_loss

We've made it the training method! 

SGD makes training easy: loop over minibatches; update the weights with a scalar multiple of the gradient; stop when you are satisfied that further improvements can no longer be made without overfitting. I have implemented three stopping criteria. First, if the magnitude of the difference between the latest loss evaluation and the previous loss evaluation is smaller that ```test_loss_epsilon``` then stop. This is meant to stop training if the test loss has stabilized. A more sophisticated approach would look at the variance of some number of the most recent test loss values. Second, if the mean magnitude of all the weight gradients is smaller than ```grad_mag_epsilon``` then stop. Certainly if all the graidents are small then we should be close to an extrema and further SGD updates will be ineffectual. Note that this doesn't really address the well-known vanishing/exploding gradients problem in RNNs. An approach for dealing with that is to track the singular values of the weight matrices. Third, if training exceeds ```max_seconds``` seconds in total runtime then stop. 

The following ```train``` method looks a bit more involved than it actually is. The core work of the method was described in the previous paragraph. It has been exploded a bit because I added status logging and am recording intermediate data (saving the mean gradient magnitudes and all the loss evaluations).

In [15]:
def train(self,
          eta,
          n_minibatches,
          test_loss_epsilon,
          grad_mag_epsilon,
          max_seconds
          ) : 
    """ Perform a training session on the model

        Input : 

        eta               - learning rate for SGD updates
        n_minibatches     - number of minibatches to train over in this session
        test_loss_epsilon - stop training if test loss falls below this
        grad_mag_epsilon  - stop training if any of the mean gradient magnitudes 
                            falls below this
        max_seconds       - stop training after this many seconds

        Output : 

        stopping_code     - string indicating why training stopped

    """

    self.logger.info("training")
    self.logger.info("eta               = %17.17f"%eta)
    self.logger.info("test_loss_epsilon = %17.17f"%test_loss_epsilon)
    self.logger.info("grad_mag_epsilon  = %17.17f"%grad_mag_epsilon)
    self.logger.info("max_seconds       = %17.17f"%max_seconds)

    # the stopping code
    stopping_code = 'Reached max number of minibatches (%d)'%n_minibatches

    # compile the model if need be

    self.model.compile_loss()
    self.model.compile_sgd_update()

    start = time.perf_counter()

    cnt = 0
    for i in range(self.minibatch_i,self.minibatch_i+n_minibatches) : 

        # get the minibatch

        i0 = (i*self.minibatch_size)%self.training_data_size
        i1 = ((i+1)*self.minibatch_size)%self.training_data_size

        if i1 < i0 :
            # we reached the end of the data
            i1 = self.training_data_size

        cnt += 1
        self.logger.info('minibatch %d of %d, training indices [%d, %d]'%(cnt,n_minibatches,i0,i1))

        minibatch = self.training_data[self.training_indices[i0:i1]]

        # compute an update

        sgd_i = float(self.minibatch_i)
        self.minibatch_i += 1

        self.logger.info('sgd update')

        self.model.sgd_update(minibatch,minibatch,eta,sgd_i)

        # save gradient magnitudes

        self.m_dV_mag.append(self.model.m_dV_mag.get_value())
        self.m_dU_mag.append(self.model.m_dU_mag.get_value())
        self.m_dW_mag.append(self.model.m_dW_mag.get_value())
        self.m_db_mag.append(self.model.m_db_mag.get_value())

        self.logger.info('mean dV magnitude = %17.17f'%self.m_dV_mag[-1])
        self.logger.info('mean dU magnitude = %17.17f'%self.m_dU_mag[-1])
        self.logger.info('mean dW magnitude = %17.17f'%self.m_dW_mag[-1])
        self.logger.info('mean db magnitude = %17.17f'%self.m_db_mag[-1])

        # check convergence criteria

        if i % self.eval_test_loss_stride == 0 : 
            # evaluate the test loss and check for 'convergence'

            self.logger.info('evaluating test loss')

            # accumulate the loss

            prev_test_loss = self.test_loss[-1][1]

            self.test_loss.append((self.minibatch_i,
                                   self.accumulate_loss()))

            self.logger.info('test loss = %17.17f'%self.test_loss[-1][1])

            # save the weights and biases if the loss is the best yet
            if self.test_loss[-1][1] >= self.best_test_loss[1] : 

                self.best_V = self.model.V.get_value()
                self.best_U = self.model.U.get_value()
                self.best_W = self.model.W.get_value()
                self.best_b = self.model.b.get_value()

                self.best_test_loss = (self.test_loss[-1][0],
                                       self.test_loss[-1][1])

            if np.fabs(prev_test_loss - self.test_loss[-1][1]) < test_loss_epsilon : 

                # record that we are stopping because of training loss
                # stabilization
                stopping_code = 'Training loss stabilized'

                break

        if (self.m_dV_mag[-1] < grad_mag_epsilon
                and self.m_dU_mag[-1] < grad_mag_epsilon
                and self.m_dW_mag[-1] < grad_mag_epsilon
                and self.m_db_mag[-1] < grad_mag_epsilon) : 
            # the average gradient magnitudes are small so let's give up

            self.logger.info('gradient magnitude convergence')

            # accumulate the loss to see if we are better than the 
            # current best

            if i % self.eval_test_loss_stride != 0 :

                self.test_loss.append((self.minibatch_i,
                                       self.accumulate_loss()))

            # save the weights and biases if the loss is the best yet
            if self.test_loss[-1][1] >= self.best_test_loss[1] : 

                self.best_V = self.model.V.get_value()
                self.best_U = self.model.U.get_value()
                self.best_W = self.model.W.get_value()
                self.best_b = self.model.b.get_value()

                self.best_test_loss = (self.test_loss[-1][0],
                                       self.test_loss[-1][1])

            # record that we stopped because of gradient magnitudes
            stopping_code = 'Gradient magnitudes < %f'%grad_mag_epsilon

            break

        if i1 == self.training_data_size :
            # shuffle the data for the next epoch
            self.logger.info('epoch complete')
            np.random.shuffle(self.training_indices)

        # check if we have been running too long
        elapsed = time.perf_counter() - start
        self.logger.info('elpased time = %fs'%elapsed)
        if elapsed > max_seconds : 

            stopping_code = "Exceeded time limit of %fs. Elapsed = %fs"%(max_seconds,elapsed)

            break

    elapsed = time.perf_counter() - start

    self.training_minibatches.append(self.minibatch_i)
    self.training_times.append(elapsed)

    self.logger.info('stopping code = %s'%stopping_code)
    self.logger.info('training run time %fs'%elapsed)
    self.logger.info('total elapsed training run time %fs'%elapsed)
    self.logger.info('ended at minibatch %d of %d'%(cnt,n_minibatches))
    self.logger.info('total minibatches %d'%self.minibatch_i)
    self.logger.info('test loss last evaluated at minibatch %d'%self.test_loss[-1][0])
    self.logger.info('last test loss    = %17.17f'%self.test_loss[-1][1])
    self.logger.info('mean dV magnitude = %17.17f'%self.m_dV_mag[-1])
    self.logger.info('mean dU magnitude = %17.17f'%self.m_dU_mag[-1])
    self.logger.info('mean dW magnitude = %17.17f'%self.m_dW_mag[-1])
    self.logger.info('mean db magnitude = %17.17f'%self.m_db_mag[-1])

    return stopping_code

# bind the method to the class
word_embedder.train = train

Everything is in place to run some training iterations on the very small example data set in sentences_sml.npz; it consists of 1000 sentences with 2305 unique words. Here is the embedder initialization:

In [16]:
sentences_path = r'./sentences_sml.npz'
n_vocabulary = 2300
n_training_data = 800
embedding_space_dim = 2
minibatch_size = 400
eval_test_loss_stride = 2
n_test_loss_batches = 1
nof1_dtype = 'int16'
model_dtype = 'float32'
shuffle_seed = 123
model_seed = 456
embedder_seed = 789

embedder = word_embedder.init_from_sentences_on_disk(sentences_path,
                                                     n_vocabulary,
                                                     n_training_data,
                                                     embedding_space_dim,
                                                     minibatch_size,
                                                     eval_test_loss_stride,
                                                     n_test_loss_batches,
                                                     nof1_dtype,
                                                     model_dtype,
                                                     shuffle_seed,
                                                     model_seed,
                                                     embedder_seed)

INFO:reading sentences from ./sentences_sml.npz
INFO:computing one-of-n indices
INFO:initializing embedding model
INFO:init from sentences elapsed time = 0.384053s


I've asked for the training data to have 800 sentences, leaving 200 sentences for the test set. I've set the number of embedding space dimensions to 2, which is very small but good enough as far as this notebook is concerned since I just want to show the code running a few training iterations. A minibatch will contain 400 sentences, so two minibatches is one epoch (a complete pass through the training data). By setting ```eval_test_loss_stride = 2```, I am requesting that the loss be computed every other training iteration (every 2 minibatchs). The variable ```n_test_loss_batches``` determines across how many batches the loss should be accumulated - as noted above this is a layover from the original  inefficient implementation of ```rnn```.

Ok! We are ready to run some training iterations:

In [17]:
eta = np.array(0.1,dtype=model_dtype)
n_minibatches = 4
test_loss_epsilon = 0.0001
grad_mag_epsilon = 0.0001
max_seconds = 60.

embedder.train(eta,
               n_minibatches,
               test_loss_epsilon,
               grad_mag_epsilon,
               max_seconds)

INFO:training
INFO:eta               = 0.10000000149011612
INFO:test_loss_epsilon = 0.00010000000000000
INFO:grad_mag_epsilon  = 0.00010000000000000
INFO:max_seconds       = 36000.00000000000000000
INFO:minibatch 1 of 4, training indices [0, 400]
INFO:sgd update
INFO:mean dV magnitude = 1.29920578002929688
INFO:mean dU magnitude = 0.11843164265155792
INFO:mean dW magnitude = 0.17303091287612915
INFO:mean db magnitude = 0.16622026264667511
INFO:evaluating test loss
INFO:accumulating loss
INFO:batch 1 of 1
INFO:loss accumulation complete
INFO:test loss = -8.29797744750976562
INFO:elpased time = 2.708264s
INFO:minibatch 2 of 4, training indices [400, 800]
INFO:sgd update
INFO:mean dV magnitude = 1.29996919631958008
INFO:mean dU magnitude = 0.10756405442953110
INFO:mean dW magnitude = 0.15645948052406311
INFO:mean db magnitude = 0.15097820758819580
INFO:epoch complete
INFO:elpased time = 5.080980s
INFO:minibatch 3 of 4, training indices [0, 400]
INFO:sgd update
INFO:mean dV magnitude = 1.2

'Reached max number of minibatches (4)'

And that's it! It's time to see what happens when I feed this thing 5 million sentences!